# Data Science Exercise-1, fuzzy record matching program from the given input data sets.

In [36]:
#Required PACKAES to install
#The Levenshtein distance is a number that tells you how different two strings are. 
#The higher the number, the more different the two strings are.
!conda install -c conda-forge python-levenshtein
!pip install fuzzymatcher
!pip install recordlinkage

Solving environment: ...working... done

# All requested packages already installed.



In [37]:
#Pre requisite to run the python notebook
#"sqlite-dll-win64-x64-3370200.zip" this dll need to be downloaded from 'https://www.sqlite.org/download.html'
#From the above archived file 'sqlite3.dll' need to copy to local python bin folder example#'C:\Users\HP\anaconda3\Library\bin'
#Once dll is copied to local machine, windows machine need to restart.

In [38]:
#Fuzzymatches uses sqlite3's Full Text Search to find potential matches.
#It then uses probabilistic record linkage to score matches.
#Finally it outputs a list of the matches it has found and associated score.

fuzzymatcher uses a number of components, each one of which can be re-written or adapted by the user:

data_preprocessor: Responsible for normalising strings, removing punctuation etc.
datagetter: Responsible for finding a list of possible matches for each df_left record in df_right
scorer: Responsible for computing a match score, given a record from dataframe_left and dataframe_right respectively.

In [39]:
#Importing required python libraries
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage

In [40]:
#Created two csv files from the given input file to separate query and refernce data.
#Re indexed the reference dataframe with '0' since Fuzzymatches linkage works on Zero based indexing
#Creating reference/Query dataframes from the above created csv files.
Query_df = pd.read_csv('C:\\Sanjeev\\Juniper\\Query.csv')
reference_df = pd.read_csv('C:\\Sanjeev\\Juniper\\reference.csv')

In [41]:
#Created data frames
reference_df.shape, Query_df.shape

((782, 5), (82, 7))

In [42]:
#Dataframe rows
reference_df.head()

,id,name,address,city,cuisine
0,0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american
1,1,art's delicatessen,12224 ventura blvd.,studio city,american
2,2,hotel bel-air,701 stone canyon rd.,bel air,californian
3,3,cafe bizou,14016 ventura blvd.,sherman oaks,french
4,4,campanile,624 s. la brea ave.,los angeles,american


In [43]:
# Selecting common Columns to match reference and query dataframes
left_on = ["name","address","city", "cuisine"]
right_on = ["name","address","city", "cuisine"]

In [44]:
#importing fts4 virtual table from sqlite3
import sqlite3
from sqlite_fts4 import register_functions

conn = sqlite3.connect(":memory:")
register_functions(conn)

In [45]:
#Enabling the FTS4/FTS5 SQLITE virtual tables for sqlite seach operation
import sqlite3, platform

print('sqlite {}'.format(sqlite3.version_info))
print('sqlite {}'.format(sqlite3.sqlite_version_info))
print('sqlite {}'.format(sqlite3.__file__))

db = sqlite3.connect(':memory:') # memory stream
cursor = db.cursor()

results = cursor.execute("pragma compile_options")

for r in results:
    print(r)

print()
print("Operating System: {} {}".format(platform.system(), platform.release()))
print("Platform: {} {}".format(platform.python_implementation(),platform.python_version()))
print("SQLite: {}".format(sqlite3.sqlite_version))

sqlite (2, 6, 0)
sqlite (3, 37, 2)
sqlite C:\Users\HP\anaconda3\lib\sqlite3\__init__.py
('ATOMIC_INTRINSICS=0',)
('COMPILER=msvc-1500',)
('DEFAULT_AUTOVACUUM',)
('DEFAULT_CACHE_SIZE=-2000',)
('DEFAULT_FILE_FORMAT=4',)
('DEFAULT_JOURNAL_SIZE_LIMIT=-1',)
('DEFAULT_MMAP_SIZE=0',)
('DEFAULT_PAGE_SIZE=4096',)
('DEFAULT_PCACHE_INITSZ=20',)
('DEFAULT_RECURSIVE_TRIGGERS',)
('DEFAULT_SECTOR_SIZE=4096',)
('DEFAULT_SYNCHRONOUS=2',)
('DEFAULT_WAL_AUTOCHECKPOINT=1000',)
('DEFAULT_WAL_SYNCHRONOUS=2',)
('DEFAULT_WORKER_THREADS=0',)
('ENABLE_BYTECODE_VTAB',)
('ENABLE_COLUMN_METADATA',)
('ENABLE_DBPAGE_VTAB',)
('ENABLE_DBSTAT_VTAB',)
('ENABLE_FTS3',)
('ENABLE_FTS4',)
('ENABLE_FTS5',)
('ENABLE_GEOPOLY',)
('ENABLE_JSON1',)
('ENABLE_MATH_FUNCTIONS',)
('ENABLE_PREUPDATE_HOOK',)
('ENABLE_RTREE',)
('ENABLE_SESSION',)
('ENABLE_STMTVTAB',)
('MALLOC_SOFT_LIMIT=1024',)
('MAX_ATTACHED=10',)
('MAX_COLUMN=2000',)
('MAX_COMPOUND_SELECT=500',)
('MAX_DEFAULT_PAGE_SIZE=8192',)
('MAX_EXPR_DEPTH=1000',)
('MAX_FUNCTION_AR

In [46]:
# perform the match using fuzzymatcher method
# this method is doing best match score based on similarity for the given dataframe columns.
matched_results = fuzzymatcher.fuzzy_left_join(Query_df,reference_df,right_on,
                                               left_on,left_id_col='id',right_id_col='id')

In [47]:
matched_results.columns

Index(['best_match_score', '__id_left', '__id_right', 'id_left', 'name_left',
       'address_left', 'city_left', 'cuisine_left', 'reference_id',
       'score(optional)', 'id_right', 'name_right', 'address_right',
       'city_right', 'cuisine_right'],
      dtype='object')

In [48]:
cols = ["best_match_score","__id_left","__id_right","name_left","name_right","address_left","address_right","city_left","city_right","cuisine_left","cuisine_right"]
matched_results.shape

(82, 15)

In [49]:
#Renaming the 'best_match_score' dataframe column to 'score'
matched_results.rename(columns={"best_match_score":"score"}, inplace=True)

#Converting matched score to redable format using apply/map with lamda function
#lamda function is efficient inline function
matched_results['score'] = matched_results['score'].apply(lambda score: score * 100)
#matched_results['score'] = matched_results['score'].map(lambda score: score * 100)

In [50]:
#rounding off the score value
matched_results=matched_results.round({'score': 0})
matched_results.head(2)

,score,__id_left,__id_right,id_left,name_left,address_left,city_left,cuisine_left,reference_id,score(optional),id_right,name_right,address_right,city_right,cuisine_right
0,38.0,1001,30,1001,chanterelle,2 harrison st.,new york city,french (new),31.0,70.0,30,chanterelle,2 harrison st. near hudson st.,new york,american
75,80.0,1002,31,1002,daniel,20 e. 76th st.,new york city,french (new),32.0,90.0,31,daniel,20 e. 76th st.,new york,french


In [51]:
#For Perfect match the score is 100. applying the same condition to entire dataframe
matched_results['score'] = matched_results['score'].apply(lambda score: 100 if score >= 100 else score)

In [52]:
#Renaming of dataframe columns to a redable format columns
matched_results.rename(columns={"best_match_score":"score","__id_right":"reference_id","name_left":"query_name",
"address_left":"query_address","city_left":"query_city","cuisine_left":"query_cuisine",
"name_right":"reference_name",
"address_right":"reference_address","city_right":"reference_city","cuisine_right":"reference_cuisine"}, inplace=True)

In [53]:
#Selection of required dataframe columns only
matched_results=matched_results[['query_name','query_address', 'query_city', 'query_cuisine','reference_name', 'reference_address',
       'reference_city', 'reference_cuisine','reference_id','score']]

In [54]:
matched_results.shape

(82, 11)

In [55]:
#Removal of duplicate columns
matched_results = matched_results.loc[:,~matched_results.columns.duplicated()]
matched_results.shape

(82, 10)

In [56]:
#Calculating Partial Accuracy of the model with threshold of 30 or moe
count=matched_results.loc[matched_results['score'] > 30.0].shape[0]
tcount=matched_results.shape[0]
PartialAccuracy=(count/tcount)*100
round(PartialAccuracy,0)

78.0

In [57]:
#Calculating the Accuracy for a threshold of 50 and above matching score
count=matched_results.loc[matched_results['score'] > 50.0].shape[0]
tcount=matched_results.shape[0]
Accuracy=(count/tcount)*100
round(Accuracy,0)

63.0

In [58]:
#query_name,  query_address, query_city, query_cuisine  are Query dataframe columns
#reference_name reference_address reference_city reference_cuisine are reference dataframe columns
#reference_id, score  are fuzzymatch reference_id pulled from reference_df dataframe and 'score' is fuzzy match score
matched_results

,query_name,query_address,query_city,query_cuisine,reference_name,reference_address,reference_city,reference_cuisine,reference_id,score
0,chanterelle,2 harrison st.,new york city,french (new),chanterelle,2 harrison st. near hudson st.,new york,american,30,38.0
75,daniel,20 e. 76th st.,new york city,french (new),daniel,20 e. 76th st.,new york,french,31,80.0
107,dawat,210 e. 58th st.,new york city,indian,sparks steak house,210 e. 46th st.,new york city,steakhouses,640,6.0
116,felidia,243 e. 58th st.,new york city,italian,felidia,243 e. 58th st.,new york,italian,33,79.0
147,four seasons,99 e. 52nd st.,new york city,american (new),arcadia,21 e. 62nd st.,new york city,american (new),602,14.0
...,...,...,...,...,...,...,...,...,...,...
2790,mifune,1737 post st.,san francisco,japanese,mifune japan center kintetsu building,1737 post st.,san francisco,asian,107,21.0
2840,plumpjack cafe,3127 fillmore st.,san francisco,american (new),trio cafe,1870 fillmore st.,san francisco,american,748,6.0
2844,postrio,545 post st.,san francisco,californian,postrio,545 post st.,san francisco,american,109,61.0
2896,ritz-carlton dining room (san francisco),600 stockton st.,san francisco,french (new),ritz-carlton restaurant and dining room,600 stockton st.,san francisco,american,110,72.0


# Fuzzy matching Batch Method-1:

In [59]:
##Local file path to save the results
matched_results.to_csv('C:\\Sanjeev\\Juniper\\match_query02.csv')

In [60]:
#Interactive method to test the matching strings
from fuzzywuzzy import fuzz
fuzz.ratio('daniel20 e.76th st. new york city french (new)','daniel20 e. 76th st. new york french')
#fuzz.ratio(Query_df['name'][0], Query_df['name'][0])

85

# Fuzzy matching Batch Method-2 (Improved method)

In [61]:
Query_df.columns, reference_df.columns

(Index(['id', 'name', 'address', 'city', 'cuisine', 'reference_id',
        'score(optional)'],
       dtype='object'),
 Index(['id', 'name', 'address', 'city', 'cuisine'], dtype='object'))

In [62]:
#Combining "name,address,city,cuisine" columns of Query/refernce data frame to create new column.
Query_df['NameAddressCityCuisine']=Query_df['name']+Query_df['address']+Query_df['city']+Query_df['cuisine']
reference_df['NameAddressCityCuisine']=reference_df['name']+reference_df['address']+reference_df['city']+reference_df['cuisine']

In [63]:
#Addition of new column 'NameAddressCityCuisine' for Query_df and reference_df.
reference_df.head(2)

,id,name,address,city,cuisine,NameAddressCityCuisine
0,0,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american,arnie morton's of chicago 435 s. la cienega bl...
1,1,art's delicatessen,12224 ventura blvd.,studio city,american,art's delicatessen 12224 ventura blvd. studio ...


In [64]:
#Importing process and fuzz packages.
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [65]:
#Python method to extract the match and its score from reference dataframe.
#This method is more effective and efficient in terms of accuracy/performance i.e accuracy#80%
def fuzzy_match(x, choices, scorer, cutoff):
    return process.extractOne(x, choices=choices, scorer=scorer, score_cutoff=cutoff)

FuzzyWuzzyResults = Query_df.loc[:,'NameAddressCityCuisine'].apply(fuzzy_match, args=(reference_df.loc[:,'NameAddressCityCuisine'], fuzz.token_set_ratio,80))
#FuzzyWuzzyResults = Query_df.loc[:,'NameAddressCityCuisine'].apply(fuzzy_match, args=(reference_df.loc[:,'NameAddressCityCuisine'], fuzz.token_sort_ratio,80))

# Results format:
#Query datafarme combined column string (NameAddressCityCuisine), Match score, Record linkage reference from Reference dataframe.
#('chanterelle 2 harrison st.  near hudson st. new york american', 85, 30)

In [66]:
#Local file path to save the results for validation
FuzzyWuzzyResults.to_csv('C:\\Sanjeev\\Juniper\\FuzzyWuzzyResults.csv')

# Fuzzy matching Interactive Method-1:

In [67]:
#Interactive method for Exact match with different case in string.
fuzz.token_sort_ratio('chanterelle 2 Harrison st.  near hudson st. new york american', 'Chanterelle 2 harrison st.  Near hudson st. new york american')

100

# Fuzzy matching Interactive Method-2:

In [68]:
#Exact match for different case and in different order of words.
fuzz.token_sort_ratio('chanterelle 2 Harrison st.  near hudson st. new york american', 'new york american Chanterelle 2 harrison st.  Near hudson st. ')

100